In [1]:
from __future__ import print_function
import sqlite3
import paho.mqtt.client as mqtt
import csv
import datetime
import os
import json
import requests
import sys
import msgpack
from kafka import KafkaProducer
from kafka import SimpleProducer, KafkaClient
#basepath = '/root/sftp/volumes/saqn/

In [4]:
import threading, logging, time
import multiprocessing

from kafka import KafkaConsumer, KafkaProducer

kafkaUrl = "smartaqnet-dev01.teco.edu"
#kafkaUrl = "smartaqnet.teco.edu"
class Producer(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.stop_event = threading.Event()
        
    def stop(self):
        self.stop_event.set()

    def run(self):
        producer = KafkaProducer(bootstrap_servers=kafkaUrl+':9092')

        while not self.stop_event.is_set():
            producer.send('Baka', str(time.time()))
            
            time.sleep(2)

        producer.close()

class Consumer(multiprocessing.Process):
    def __init__(self):
        
        multiprocessing.Process.__init__(self)
        self.stop_event = multiprocessing.Event()
        
    def stop(self):
        self.stop_event.set()
        
    def run(self):
        consumer = KafkaConsumer(bootstrap_servers= kafkaUrl + ':9092',
                                 auto_offset_reset='latest',
                                 consumer_timeout_ms=1000)#,value_deserializer=msgpack.unpackb)
        consumer.subscribe(['Observations'])

        while not self.stop_event.is_set():
            for message in consumer:
                print(message)
                
                if self.stop_event.is_set():
                    break

        consumer.close()

In [5]:
tasks = [
        #Producer(),
        Consumer()
]
for t in tasks:
        t.start()
time.sleep(10)
for t in tasks:
        t.stop()

ConsumerRecord(topic=u'Observations', partition=0, offset=1885047, timestamp=1527692797189, timestamp_type=0, key='edu.teco.wang/IMOERS40/Datastreams/Air-Temperature-IMOERS40', value='{"Datastream": {"@iot.id": "edu.teco.wang/IMOERS40/Datastreams/Air-Temperature-IMOERS40", "unitOfMeasurement": {"definition": null, "symbol": null, "name": null}}, "FeatureOfInterest@iot.navigationLink": "http://localhost:8080/FROST-Server/v1.0/Observations(\'0c851c5c-641b-11e8-8042-935937257529\')/FeatureOfInterest", "phenomenonTime": "2015-12-31T22:45:00.000Z", "@iot.id": "0c851c5c-641b-11e8-8042-935937257529", "@iot.selfLink": "http://localhost:8080/FROST-Server/v1.0/Observations(\'0c851c5c-641b-11e8-8042-935937257529\')", "result": "6.9", "FeatureOfInterest": {"@iot.id": "edu.teco.wang/IMOERS40/FoI/IMOERS40/51.455:6.657"}, "resultTime": "2015-12-31T22:45:00.000Z", "Datastream@iot.navigationLink": "http://localhost:8080/FROST-Server/v1.0/Observations(\'0c851c5c-641b-11e8-8042-935937257529\')/Datastream

In [4]:
time.time()

1527681003.223547

In [12]:
for t in tasks:
        t.stop()